<a href="https://colab.research.google.com/github/adityasinghji1086/-sparkprojects/blob/main/sparkproject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
 !mkdir -p data
!mkdir -p output/parquet_data


In [ ]:
!ls


data  output  sample_data


In [ ]:
 %%writefile data/employees.csv
Emp_id,Name,Dept,Salary,City,Experience
1,Amit,HR,30000,Delhi,2
2,Rahul,HR,40000,Mumbai,4
3,Neha,IT,50000,Bangalore,3
4,Pooja,IT,60000,Bangalore,5
5,Rohit,Sales,35000,Delhi,3
6,Karan,Sales,45000,Mumbai,6
7,Simran,HR,38000,Delhi,3
8,Ankit,IT,70000,Pune,7
9,Priya,Sales,32000,Jaipur,2
10,Deepak,IT,55000,Bangalore,4


Writing data/employees.csv


In [ ]:
 from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("EmployeeAnalyticsCase") \
    .getOrCreate()


In [ ]:
 df = spark.read.csv(
    "data/employees.csv",
    header=True,
    inferSchema=True
)

df.show()


+------+------+-----+------+---------+----------+
|Emp_id|  Name| Dept|Salary|     City|Experience|
+------+------+-----+------+---------+----------+
|     1|  Amit|   HR| 30000|    Delhi|         2|
|     2| Rahul|   HR| 40000|   Mumbai|         4|
|     3|  Neha|   IT| 50000|Bangalore|         3|
|     4| Pooja|   IT| 60000|Bangalore|         5|
|     5| Rohit|Sales| 35000|    Delhi|         3|
|     6| Karan|Sales| 45000|   Mumbai|         6|
|     7|Simran|   HR| 38000|    Delhi|         3|
|     8| Ankit|   IT| 70000|     Pune|         7|
|     9| Priya|Sales| 32000|   Jaipur|         2|
|    10|Deepak|   IT| 55000|Bangalore|         4|
+------+------+-----+------+---------+----------+



In [ ]:
 dept_total_salary = df.groupBy("Dept") \
    .sum("Salary") \
    .withColumnRenamed("sum(Salary)", "Total_Salary")

dept_total_salary.show()


+-----+------------+
| Dept|Total_Salary|
+-----+------------+
|Sales|      112000|
|   HR|      108000|
|   IT|      235000|
+-----+------------+



In [ ]:
dept_avg_salary = df.groupBy("Dept") \
    .avg("Salary") \
    .withColumnRenamed("avg(Salary)", "Avg_Salary")

dept_avg_salary.show()


+-----+------------------+
| Dept|        Avg_Salary|
+-----+------------------+
|Sales|37333.333333333336|
|   HR|           36000.0|
|   IT|           58750.0|
+-----+------------------+



In [ ]:
 from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

windowSpec = Window.partitionBy("Dept").orderBy(desc("Salary"))

top_employee = df.withColumn("rank", rank().over(windowSpec)) \
    .filter("rank = 1") \
    .select("Emp_id", "Name", "Dept", "Salary")

top_employee.show()


+------+-----+-----+------+
|Emp_id| Name| Dept|Salary|
+------+-----+-----+------+
|     2|Rahul|   HR| 40000|
|     8|Ankit|   IT| 70000|
|     6|Karan|Sales| 45000|
+------+-----+-----+------+



In [ ]:
dept_total_salary.write.mode("overwrite") \
    .parquet("output/parquet_data/dept_total_salary")

dept_avg_salary.write.mode("overwrite") \
    .parquet("output/parquet_data/dept_avg_salary")

top_employee.write.mode("overwrite") \
    .parquet("output/parquet_data/top_employee")


In [ ]:
!ls output/parquet_data


dept_avg_salary  dept_total_salary  top_employee


In [ ]:
spark.read.parquet(
    "output/parquet_data/top_employee"
).show()


+------+-----+-----+------+
|Emp_id| Name| Dept|Salary|
+------+-----+-----+------+
|     2|Rahul|   HR| 40000|
|     8|Ankit|   IT| 70000|
|     6|Karan|Sales| 45000|
+------+-----+-----+------+

